In [ ]:
workdir_ = '/glade/work/juliob/'
import sys
#######################################
# Leave this for now. But it should change to better
# method as here:
# import os
# module_a_dir = os.path.dirname(os.path.abspath(__file__))
# utils_path = os.path.join(module_a_dir, '..', 'Utils')
# sys.path.append(utils_path)
# print( f" a path added in {__name__} {utils_path} ")
########################################
sys.path.append(workdir_ + 'myPythonTools/GenlTools/')
sys.path.append(workdir_ + 'myPythonTools/Utils/')
#sys.path.append(workdir_ + 'PyRegridding/Regridder/')
sys.path.append(workdir_ + 'PyRegridding/Utils/')

# Own local packages
import AveragingUtils as Av
#import VertRegridFlexLL as Vrg  # This is toxic for some reason
import PlotUtil as Pu
import validation_data as Val
import var_A_x_B as vAB
import MakePressures as MkP
import GridUtils as GrU

# The usual
from datetime import date
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import matplotlib.tri as tri

# Cartopy for pretty maps
import cartopy
import cartopy.crs as ccrs
import cartopy.feature as cfeature

# Some other useful packages 
import importlib
import copy
import time
import cftime

importlib.reload( Pu )
importlib.reload(Av)
importlib.reload(Val)
importlib.reload(vAB)
importlib.reload(MkP)


In [ ]:
import matplotlib
print(matplotlib.__version__)
print(cartopy.__version__)

In [ ]:
#####
exp_Val='ERA-5'

#user_C,user_X='juliob','juliob'

#subd='hist'
subd='regridded'

#exp_C,user_C = 'test3.02.2' , 'juliob'
#exp_C,user_C = 'fmthist_MM_x21_2' , 'juliob'
#exp_X,user_X = 'fmthist_MM_ncpl192' , 'juliob'
#exp_C,user_C = 'POLARRES_FMT_x01_C' , 'juliob'
#exp_X,user_X = 'fmtndg2000_MM_control' , 'juliob'

trim_years_c, nyr_max_c = False, 1000
trim_years_x, nyr_max_x = False, 1000

exp_C,user_C = 'fmthist_MM_control' , 'juliob'
#exp_C,user_C = 'fmthist_MM_x21_2' , 'juliob'
#exp_X,user_X = 'fmthist_MM_x21_6' , 'juliob'
#exp_X,user_X = 'POLARRES_FMT_x01_C' , 'juliob'
exp_X,user_X = 'fmthist_MM_ncpl192' , 'juliob'

#exp_C,user_C = 'WACCM FWHIST CMIP6','CMIP6'
###

if (user_C == 'juliob' ):
    path_C = '/glade/derecho/scratch/juliob/archive/'+exp_C+'/atm/'+ subd +'/'+exp_C+'.cam.h0.*.nc' 
elif (user_C == 'tilmes' ):
    path_C = '/glade/derecho/scratch/tilmes/archive/'+exp_C+'/atm/hist/'+exp_C+'.cam.h0.*.nc' 
elif (user_C == 'pel' ):
    path_C = '/glade/derecho/scratch/pel/archive/'+exp_C+'/atm/hist/'+exp_C+'.cam.h0.*.nc' 
elif (user_C == 'juliob_run' ):
    path_C = '/glade/derecho/scratch/juliob/'+exp_C+'/run/'+exp_C+'.cam.h0.*.nc' 
elif (user_C == 'CMIP6' ):
    dir='/glade/campaign/collections/cmip/CMIP6/timeseries-cmip6/f.e21.FWHISTBgcCrop.f09_f09_mg17.CMIP6-AMIP-WACCM.001/atm/proc/tseries/month_1/'
    path_C=dir+ 'f.e21.FWHISTBgcCrop.f09_f09_mg17.CMIP6-AMIP-WACCM.001.cam.h0.U.195001-201412.nc'


if (user_X == 'juliob' ):
    path_X = '/glade/derecho/scratch/juliob/archive/'+exp_X+'/atm/'+ subd +'/'+exp_X+'.cam.h0.*.nc' 
elif (user_X == 'tilmes' ):
    path_X = '/glade/derecho/scratch/tilmes/archive/'+exp_X+'/atm/hist/'+exp_X+'.cam.h0.*.nc' 
elif (user_X == 'pel' ):
    path_X = '/glade/derecho/scratch/pel/archive/'+exp_X+'/atm/hist/'+exp_X+'.cam.h0.*.nc' 
elif (user_X == 'juliob_run' ):
    path_X = '/glade/derecho/scratch/juliob/'+exp_X+'/run/'+exp_X+'.cam.h0.*.nc' 



###

Dc = xr.open_mfdataset( path_C ,data_vars='different', coords='different' )
Dx = xr.open_mfdataset( path_X ,data_vars='different', coords='different' )

In [ ]:
########################
# Some massaging
########################
trim_years_c, nyr_max_c = True, 1000
trim_years_x, nyr_max_x = True, 1000

if ( trim_years_c==True ):
    nyr=np.minimum( nyr_max_c*12 , 12*(len(Dc.time)//12) )
    print(f"Discarding last {len(Dc.time)-nyr} months of {exp_C}" )
    Dc=Dc.isel(time=np.arange(nyr))

if ( trim_years_x==True ):
    nyr=np.minimum( nyr_max_x*12 , 12*(len(Dx.time)//12) )
    print(f"Discarding last {len(Dx.time)-nyr} months of {exp_X}" )
    Dx=Dx.isel(time=np.arange(nyr))


##
#ps_c =Dc.PS.values
lon_c=Dc.lon.values
lat_c=Dc.lat.values
lev_c=Dc.lev.values
zlev_c=-7.0*np.log( lev_c/1_000. )

hyai_c=Dc.hyai.values
hybi_c=Dc.hybi.values
hyam_c=Dc.hyam.values
hybm_c=Dc.hybm.values

#ps_x =Dx.PS.values
lon_x=Dx.lon.values
lat_x=Dx.lat.values
lev_x=Dx.lev.values
zlev_x=-7.0*np.log( lev_x/1_000. )


In [ ]:
season='jja'
UUc_s,years_c,months_c = Av.Seasonal( ds=Dc, season=season , fld='U',return_time=True)
UUx_s,years_x,months_x = Av.Seasonal( ds=Dx, season=season , fld='U',return_time=True)

UUc_sz=np.average( UUc_s , axis=2 )
UUx_sz=np.average( UUx_s , axis=2 )

In [ ]:
####
# Validation
####
Validation_on_control_mgrid = False

if (Validation_on_control_mgrid == True):
    PSc_s = Av.Seasonal( ds=Dc, season=season , fld='PS')
    mgrid={'ps':PSc_s, 'hyam':hyam_c, 'hybm':hybm_c, 'hyai':hyai_c, 'hybi':hybi_c, 'hgrid':'fv0.9x1.25' }
else:
    mgrid=False

Uval = Val.data(fld='U',season=season,mgrid=mgrid,zlev=True ) #, Years='2000' )
UUv_s , zlev_v,lat_v,lon_v = Uval['aa'], Uval['lev'], Uval['lat'], Uval['lon']

UUv_sz=np.average( UUv_s , axis=2 )

In [ ]:
fig = plt.figure(figsize=(20, 5))
clev=np.linspace( -60,140,num=21) 
dlev=np.linspace( -20,20,num=21) 
cmap='gist_ncar'

Axes1 = Pu.axes_def(n=1,nxplo=3,nyplo=1 ) 
ax1 = fig.add_axes( Axes1 )
co1=ax1.contourf(lat_c,zlev_c,UUc_sz ,levels=clev, cmap=cmap )
co2=ax1.contour(lat_c,zlev_c,UUc_sz ,levels=clev, colors='black')
ax1.set_title(f"Control <{exp_C}> {season.upper()} {years_c[0]}-{years_c[-1]}" )
ax1.set_ylim(0,82)
cb=plt.colorbar(co1)

Axes1 = Pu.axes_def(n=2,nxplo=3,nyplo=1 ) 
ax1 = fig.add_axes( Axes1 )
co1=ax1.contourf(lat_x,zlev_x,UUx_sz ,levels=clev, cmap=cmap )
co2=ax1.contour(lat_x,zlev_x,UUx_sz ,levels=clev, colors='black')
ax1.set_title(f"Test <{exp_X}> {season.upper()} {years_x[0]}-{years_x[-1]}" )
ax1.set_ylim(0,82)
cb=plt.colorbar(co1)


Axes1 = Pu.axes_def(n=3,nxplo=3,nyplo=1 ) 
ax1 = fig.add_axes( Axes1 )
co1=ax1.contourf(lat_v,zlev_v, UUv_sz ,levels=clev, cmap=cmap )
co2=ax1.contour(lat_v,zlev_v, UUv_sz ,levels=clev, colors='black')
ax1.set_title(f"Validation <{exp_Val}> {season.upper()}  " )
ax1.set_ylim(0,82)
cb=plt.colorbar(co1)


In [ ]:
season='ann'
PRCc_s,years_c,months_c = Av.Seasonal( ds=Dc, season=season , fld='PRECC',return_time=True)
PRCx_s,years_x,months_x = Av.Seasonal( ds=Dx, season=season , fld='PRECC',return_time=True)
season='ann'
PRLc_s,years_c,months_c = Av.Seasonal( ds=Dc, season=season , fld='PRECL',return_time=True)
PRLx_s,years_x,months_x = Av.Seasonal( ds=Dx, season=season , fld='PRECL',return_time=True)


In [ ]:
plt.contourf(PRCx_s)

In [ ]:
######################
# Pretty latlon plot
######################

#MapProj = ccrs.Robinson(central_longitude=180.)
MapProj = ccrs.PlateCarree(central_longitude=180.)
#MapProj = ccrs.Orthographic(central_longitude=180.,central_latitude=-90.)
DataProj = ccrs.PlateCarree()

fig = plt.figure(figsize=(20, 12))

CoastColors=['white','white','white','black']
clevs = [np.linspace(0,20,21), np.linspace(0,20,21),np.linspace(0,20,21),np.linspace(0,20,21) ]

clv=[0,.1,.3,.5,1,2,3,4,5,7,10,15]
clv=np.array([0,.1,.2,.3, .5,  1,2,3,4,5,6,7,8,9,10,12])

clevs=[clv,clv,clv,clv]

cmap0 = Pu.step_cmap( 'terrain_r', clv )
cmaps=[cmap0, cmap0, cmap0, cmap0, cmap0, ]
                    
scale=86400.*1000.
PLOTfields=[PRCc_s,PRCx_s,PRLc_s,PRLx_s]
flds=['Conv Prec','Conv. Prec.','LS Prec.','LS Prec.']
exps=[ exp_C, exp_X, exp_C, exp_X]
npo=0
for field in PLOTfields:
    npo=npo+1
    Axes = Pu.axes_def(n=npo,nxplo=2,nyplo=2 ) 

    ax1 = fig.add_axes( Axes , projection=MapProj)
    ax1.set_global()
    ax1.coastlines(resolution='110m',color='black',linewidth=2)

    AAxy = scale*field 
    print( np.max(AAxy))
    
    co1=ax1.contourf(lon_c,lat_c, AAxy ,transform=DataProj,cmap=cmaps[npo-1] ,levels=clevs[npo-1])

    cbar = plt.colorbar(co1, shrink=.6)
    #ax1.set_title( CLUBBparm + zA, fontsize=16)
    ax1.set_title( f"{season.upper()} {flds[npo-1]} {exps[npo-1]}" , fontsize=16)


